# Meetup Tech Meetup Search in Berlin

This notebook demonstrates how to use the Meetup API (OAuth 2.0) to search for tech meetups in Berlin.

## Setup Instructions

1.  **Register your OAuth Consumer Application:**
    Go to [Meetup OAuth Consumers](https://secure.meetup.com/meetup_api/oauth_consumers/) and register a new application. You will need to provide a Redirect URI. For local development, you can use `http://localhost:8080` or a similar local address.

2.  **Obtain Client ID and Client Secret:**
    After registering your application, you will be provided with a **Client ID** and a **Client Secret**. Keep these secure.

3.  **Insert Credentials Below:**
    Replace the placeholder values for `CLIENT_ID`, `CLIENT_SECRET`, and `REDIRECT_URI` in the first code cell below with your actual credentials.

## OAuth 2.0 Authorization Flow

The Meetup API uses OAuth 2.0 for authentication. This involves a few steps:
1.  **Authorization Request:** You will be redirected to Meetup's website to authorize your application.
2.  **Callback:** After authorization, Meetup will redirect you back to your specified `REDIRECT_URI` with an authorization `code`.
3.  **Token Exchange:** This `code` is then exchanged for an `access_token` and `refresh_token`.

In [ ]:
import requests
import json
import webbrowser
from urllib.parse import urlencode

# --- YOUR CREDENTIALS ---
CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'
REDIRECT_URI = 'http://localhost:8080'  # Must match the Redirect URI registered on Meetup

# --- OAuth Endpoints ---
AUTHORIZATION_URL = 'https://secure.meetup.com/oauth2/authorize'
TOKEN_URL = 'https://secure.meetup.com/oauth2/access'

# --- Step 1: Get Authorization Code ---
params = {
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': REDIRECT_URI
}
auth_url = f'{AUTHORIZATION_URL}?{urlencode(params)}'

print(f'Please open this URL in your browser to authorize the application:\n{auth_url}')
print('\nAfter authorization, you will be redirected to your REDIRECT_URI. Copy the `code` from the URL and paste it below.')

authorization_code = input('Enter the authorization code: ')

# --- Step 2: Exchange Authorization Code for Access Token ---
token_params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'grant_type': 'authorization_code',
    'redirect_uri': REDIRECT_URI,
    'code': authorization_code
}

response = requests.post(TOKEN_URL, data=token_params)
response.raise_for_status()  # Raise an exception for HTTP errors
tokens = response.json()

ACCESS_TOKEN = tokens['access_token']
REFRESH_TOKEN = tokens.get('refresh_token')

print('\nAccess Token obtained successfully!')
print(f'Access Token: {ACCESS_TOKEN}')
if REFRESH_TOKEN:
    print(f'Refresh Token: {REFRESH_TOKEN}')


## Search for Tech Meetups in Berlin

Now that we have an access token, we can use it to search for events.

We will use the Meetup API's `events` endpoint. For more details on available parameters, refer to the [Meetup API documentation](https://www.meetup.com/api/guide/#events).

In [ ]:
# --- Meetup API Events Search ---
EVENTS_API_URL = 'https://api.meetup.com/find/events'

headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

# Parameters for searching tech meetups in Berlin
# 'text' can be used to search for keywords like 'tech', 'software', 'development', etc.
# 'location' can be set to 'Berlin, Germany'
# 'topic_category' can be used to filter by specific tech categories if available (e.g., 'tech', 'programming')
# 'fields' can be used to request additional fields in the response (e.g., 'group_photo', 'venue')

search_params = {
    'text': 'tech',
    'location': 'Berlin, Germany',
    'page': 20,  # Number of events per page
    'fields': 'group_photo,venue'  # Request additional fields
}

try:
    response = requests.get(EVENTS_API_URL, headers=headers, params=search_params)
    response.raise_for_status()
    events = response.json()

    if events:
        print(f'Found {len(events)} tech meetups in Berlin:')
        for event in events:
            print(f"\nEvent Name: {event.get('name')}")
            print(f"Link: {event.get('link')}")
            print(f"Date: {event.get('local_date')} {event.get('local_time')}")
            if 'venue' in event and event['venue']:
                print(f"Venue: {event['venue'].get('name')}, {event['venue'].get('address_1')}, {event['venue'].get('city')}")
            if 'group' in event:
                print(f"Group: {event['group'].get('name')}")
    else:
        print('No tech meetups found in Berlin with the given criteria.')

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    if response.status_code == 401:
        print("Authentication failed. Please check your access token or re-run the OAuth flow.")
    elif response.status_code == 403:
        print("Forbidden. You might not have the necessary permissions to access this resource.")
    elif response.status_code == 404:
        print("Not Found. The API endpoint or resource might not exist.")
    else:
        print(f"Response content: {response.text}")
